In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def init_driver(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    return driver

def get_threads_in_category(driver, cat_url, max_threads=10):
    driver.get(cat_url)
    wait = WebDriverWait(driver, 20)  # wait longer just in case
    
    # Correct XenForo thread title selector
    thread_links = wait.until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.structItem-title a"))
    )
    
    threads = []
    for t in thread_links[:max_threads]:
        href = t.get_attribute("href")
        title = t.text.strip()
        threads.append((title, href))
    return threads

if __name__ == "__main__":
    driver = init_driver(headless=False)
    category_url = "https://www.diabetes.co.uk/forum/category/diabetes-discussions.1/"
    
    threads = get_threads_in_category(driver, category_url, max_threads=5)
    for title, link in threads:
        print(title, "->", link)
    
    driver.quit()


Moderator Elections 2025 -> https://www.diabetes.co.uk/forum/threads/moderator-elections-2025.209890/
Diabetes Burnout. -> https://www.diabetes.co.uk/forum/threads/diabetes-burnout.13994/
It doesn't have to turn out like that Panorama programme suggested -> https://www.diabetes.co.uk/forum/threads/it-doesnt-have-to-turn-out-like-that-panorama-programme-suggested.108923/
Crowdstrike killed the NHS app -> https://www.diabetes.co.uk/forum/threads/crowdstrike-killed-the-nhs-app.205309/
Librelink IOS issue info, “white screen of death.” ; How to resolve. -> https://www.diabetes.co.uk/forum/threads/librelink-ios-issue-info-%E2%80%9Cwhite-screen-of-death-%E2%80%9D-how-to-resolve.197946/


In [5]:
driver.get("https://www.diabetes.co.uk/forum/category/diabetes-discussions.1/")
print(driver.page_source[:1000])  # preview HTML


<html id="XF" lang="en-GB" dir="LTR" data-xf="2.3" data-app="public" data-template="forum_view" data-container-key="node-1" data-content-key="forum-1" data-logged-in="false" data-cookie-prefix="xf_" data-csrf="1759433781,ce432d05d6194eb7631215a0f7c28da5" class="has-js template-forum_view has-no-touchevents has-passiveeventlisteners has-no-hiddenscroll has-overflowanchor has-no-displaymodestandalone has-flexgap has-os-windows has-browser-chrome"><head>
	
		<link rel="amphtml" href="https://www.diabetes.co.uk/forum/category/diabetes-discussions.1/?amp=1">
	

	
	
	

	<meta charset="utf-8">
	<title>Diabetes Discussions | Diabetes Forum</title>
	<link rel="manifest" href="/forum/webmanifest.php">

	<meta http-equiv="X-UA-Compatible" content="IE=Edge">
	<meta name="viewport" content="width=device-width, initial-scale=1, viewport-fit=cover">

	
		<meta name="theme-color" content="#0f578a">
	

	<meta name="apple-mobile-web-app-title" content="DCUK">
	
		<link rel="apple-touch-icon" href="/foru

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

def init_driver(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    return webdriver.Chrome(options=options)

def get_threads_in_category(driver, cat_url, max_threads=5):
    driver.get(cat_url)
    wait = WebDriverWait(driver, 20)
    thread_links = wait.until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.structItem-title a"))
    )
    threads = []
    for t in thread_links[:max_threads]:
        threads.append((t.text.strip(), t.get_attribute("href")))
    return threads

def scrape_thread(driver, thread_url):
    driver.get(thread_url)
    wait = WebDriverWait(driver, 20)

    posts = []
    while True:
        post_elems = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.bbWrapper"))
        )
        for p in post_elems:
            text = p.text.strip()
            if text:
                posts.append(text)

        # check if there is a "next page" button
        try:
            next_btn = driver.find_element(By.CSS_SELECTOR, "a.pageNav-jump--next")
            next_url = next_btn.get_attribute("href")
            if next_url:
                driver.get(next_url)
                time.sleep(2)
                continue
        except:
            break  # no more pages

    return posts

if __name__ == "__main__":
    driver = init_driver(headless=False)
    category_url = "https://www.diabetes.co.uk/forum/category/diabetes-discussions.1/"
    
    # get some threads
    threads = get_threads_in_category(driver, category_url, max_threads=3)
    print(f"Found {len(threads)} threads")

    # open CSV
    with open("diabetes_forum_posts.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Thread Title", "Thread URL", "Post Index", "Post Text"])

        for title, link in threads:
            print(f"\nScraping: {title} -> {link}")
            posts = scrape_thread(driver, link)
            for i, post in enumerate(posts, start=1):
                writer.writerow([title, link, i, post])

    driver.quit()


Found 3 threads

Scraping: Moderator Elections 2025 -> https://www.diabetes.co.uk/forum/threads/moderator-elections-2025.209890/

Scraping: Diabetes Burnout. -> https://www.diabetes.co.uk/forum/threads/diabetes-burnout.13994/

Scraping: It doesn't have to turn out like that Panorama programme suggested -> https://www.diabetes.co.uk/forum/threads/it-doesnt-have-to-turn-out-like-that-panorama-programme-suggested.108923/


In [9]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def init_driver(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    return webdriver.Chrome(options=options)

def get_threads_in_category(driver, cat_url, max_threads=5):
    driver.get(cat_url)
    wait = WebDriverWait(driver, 20)
    thread_links = wait.until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.structItem-title a"))
    )
    threads = []
    for t in thread_links[:max_threads]:
        threads.append((t.text.strip(), t.get_attribute("href")))
    return threads

def scrape_thread(driver, thread_url):
    driver.get(thread_url)
    wait = WebDriverWait(driver, 20)

    posts = []
    while True:
        post_elems = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.bbWrapper"))
        )
        for p in post_elems:
            text = p.text.strip()
            if text:
                posts.append(text)

        # next page check
        try:
            next_btn = driver.find_element(By.CSS_SELECTOR, "a.pageNav-jump--next")
            next_url = next_btn.get_attribute("href")
            if next_url:
                driver.get(next_url)
                time.sleep(2)
                continue
        except:
            break
    return posts

if __name__ == "__main__":
    driver = init_driver(headless=False)
    category_url = "https://www.diabetes.co.uk/forum/category/diabetes-discussions.1/"
    
    threads = get_threads_in_category(driver, category_url, max_threads=3)
    print(f"Found {len(threads)} threads")

    # Make folder for saving text files
    os.makedirs("diabetes_forum_texts", exist_ok=True)

    for title, link in threads:
        print(f"\nScraping: {title} -> {link}")
        posts = scrape_thread(driver, link)

        # Clean filename (remove illegal chars)
        safe_title = "".join(c if c.isalnum() else "_" for c in title)[:50]
        file_path = os.path.join("diabetes_forum_texts", f"{safe_title}.txt")

        with open(file_path, "w", encoding="utf-8") as f:
            f.write(f"Thread: {title}\nURL: {link}\n\n")
            for i, post in enumerate(posts, start=1):
                f.write(f"Post {i}:\n{post}\n\n{'-'*40}\n\n")

        print(f"Saved: {file_path}")

    driver.quit()
    print("\nAll threads saved in 'diabetes_forum_texts/' folder")


Found 3 threads

Scraping: Moderator Elections 2025 -> https://www.diabetes.co.uk/forum/threads/moderator-elections-2025.209890/
Saved: diabetes_forum_texts\Moderator_Elections_2025.txt

Scraping: Diabetes Burnout. -> https://www.diabetes.co.uk/forum/threads/diabetes-burnout.13994/
Saved: diabetes_forum_texts\Diabetes_Burnout_.txt

Scraping: It doesn't have to turn out like that Panorama programme suggested -> https://www.diabetes.co.uk/forum/threads/it-doesnt-have-to-turn-out-like-that-panorama-programme-suggested.108923/
Saved: diabetes_forum_texts\It_doesn_t_have_to_turn_out_like_that_Panorama_pro.txt

All threads saved in 'diabetes_forum_texts/' folder


In [10]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def init_driver(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    return webdriver.Chrome(options=options)

def get_threads_in_category(driver, cat_url, max_threads=25):
    driver.get(cat_url)
    wait = WebDriverWait(driver, 20)

    threads = []
    seen = set()

    while len(threads) < max_threads:
        # Wait for thread titles to load
        thread_links = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.structItem-title a"))
        )

        for t in thread_links:
            href = t.get_attribute("href")
            title = t.text.strip()
            if href and href not in seen:
                threads.append((title, href))
                seen.add(href)
                if len(threads) >= max_threads:
                    break

        # Scroll to bottom to load more threads
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # allow time for new threads to load

    return threads

def scrape_thread(driver, thread_url):
    driver.get(thread_url)
    wait = WebDriverWait(driver, 20)

    posts = []
    while True:
        post_elems = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.bbWrapper"))
        )
        for p in post_elems:
            text = p.text.strip()
            if text:
                posts.append(text)

        # Next page check
        try:
            next_btn = driver.find_element(By.CSS_SELECTOR, "a.pageNav-jump--next")
            next_url = next_btn.get_attribute("href")
            if next_url:
                driver.get(next_url)
                time.sleep(2)
                continue
        except:
            break
    return posts

if __name__ == "__main__":
    driver = init_driver(headless=False)
    category_url = "https://www.diabetes.co.uk/forum/category/diabetes-discussions.1/"
    
    threads = get_threads_in_category(driver, category_url, max_threads=25)
    print(f"Found {len(threads)} threads")

    # Folder for saving
    os.makedirs("diabetes_forum_texts", exist_ok=True)

    for title, link in threads:
        print(f"\nScraping: {title} -> {link}")
        posts = scrape_thread(driver, link)

        # Clean filename
        safe_title = "".join(c if c.isalnum() else "_" for c in title)[:50]
        file_path = os.path.join("diabetes_forum_texts", f"{safe_title}.txt")

        with open(file_path, "w", encoding="utf-8") as f:
            f.write(f"Thread: {title}\nURL: {link}\n\n")
            for i, post in enumerate(posts, start=1):
                f.write(f"Post {i}:\n{post}\n\n{'-'*40}\n\n")

        print(f"Saved: {file_path}")

    driver.quit()
    print("\nAll threads saved in 'diabetes_forum_texts/' folder")


Found 25 threads

Scraping: Moderator Elections 2025 -> https://www.diabetes.co.uk/forum/threads/moderator-elections-2025.209890/
Saved: diabetes_forum_texts\Moderator_Elections_2025.txt

Scraping: Diabetes Burnout. -> https://www.diabetes.co.uk/forum/threads/diabetes-burnout.13994/
Saved: diabetes_forum_texts\Diabetes_Burnout_.txt

Scraping: It doesn't have to turn out like that Panorama programme suggested -> https://www.diabetes.co.uk/forum/threads/it-doesnt-have-to-turn-out-like-that-panorama-programme-suggested.108923/
Saved: diabetes_forum_texts\It_doesn_t_have_to_turn_out_like_that_Panorama_pro.txt

Scraping: Crowdstrike killed the NHS app -> https://www.diabetes.co.uk/forum/threads/crowdstrike-killed-the-nhs-app.205309/
Saved: diabetes_forum_texts\Crowdstrike_killed_the_NHS_app.txt

Scraping: Librelink IOS issue info, “white screen of death.” ; How to resolve. -> https://www.diabetes.co.uk/forum/threads/librelink-ios-issue-info-%E2%80%9Cwhite-screen-of-death-%E2%80%9D-how-to-re

In [3]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def init_driver(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(5)
    return driver

def scroll_and_collect_topics(driver, target_count=200, pause_time=2):
    """Scrolls the latest page to collect topic links."""
    driver.get("https://community.patient.info/c/heart-health-and-blood-vessels/18")
    topic_links = []

    while len(topic_links) < target_count:
        # Scroll to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)

        # Collect links
        topics = driver.find_elements(By.CSS_SELECTOR, "a.title.raw-topic-link")
        for topic in topics:
            href = topic.get_attribute("href")
            title = topic.text.strip()
            if href and (title, href) not in topic_links:
                topic_links.append((title, href))

    print(f"Collected {len(topic_links[:target_count])} topics")
    return topic_links[:target_count]

def scroll_to_load_posts(driver, target_count=9999, pause_time=1):
    """Scrolls inside a topic to load all posts."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    posts_loaded = len(driver.find_elements(By.CSS_SELECTOR, "div.topic-post"))

    while posts_loaded < target_count:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)
        posts_loaded = len(driver.find_elements(By.CSS_SELECTOR, "div.topic-post"))
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    print(f"Loaded {posts_loaded} posts in topic")

def scrape_topic_to_text(driver, title, topic_url, output_dir="output"):
    try:
        driver.get(topic_url)
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.cooked"))
        )
        scroll_to_load_posts(driver)
    except Exception as e:
        print(f"Failed to load topic {topic_url}: {e}")
        return

    # Get page title
    try:
        page_title = driver.find_element(By.CSS_SELECTOR, "h1.title").text.strip()
    except:
        page_title = title

    posts = []
    post_containers = driver.find_elements(By.CSS_SELECTOR, "div.topic-post")
    if not post_containers:
        post_containers = driver.find_elements(By.CSS_SELECTOR, "div.cooked")

    for container in post_containers:
        author = None
        timestamp = None
        content = None

        try:
            author = container.find_element(By.CSS_SELECTOR, ".creator a.username").text.strip()
        except:
            pass
        try:
            time_el = container.find_element(By.CSS_SELECTOR, "time")
            timestamp = time_el.get_attribute("datetime")
        except:
            pass
        try:
            content = container.find_element(By.CSS_SELECTOR, "div.cooked").text.strip()
        except:
            content = container.text.strip()

        posts.append({
            "author": author,
            "timestamp": timestamp,
            "content": content
        })

    # Save to .txt
    safe_title = "".join(c if c.isalnum() or c in " _-" else "_" for c in page_title)[:80]
    os.makedirs(output_dir, exist_ok=True)
    filepath = os.path.join(output_dir, f"{safe_title}.txt")

    with open(filepath, "w", encoding="utf-8") as f:
        f.write(f"Topic: {page_title}\nURL: {topic_url}\n\n")
        for idx, post in enumerate(posts, start=1):
            f.write(f"--- Post {idx} ---\n")
            f.write(f"Author: {post.get('author')}\n")
            f.write(f"Timestamp: {post.get('timestamp')}\n")
            f.write("Content:\n")
            f.write(post.get("content", "") + "\n\n")

    print(f"Wrote {len(posts)} posts to {filepath}")

def scrape_all_latest(target_topics=200):
    driver = init_driver(headless=False)  # Set headless=True to run without UI
    try:
        topic_links = scroll_and_collect_topics(driver, target_count=target_topics)
        for idx, (title, href) in enumerate(topic_links, start=1):
            print(f"Scraping topic {idx}/{len(topic_links)}: {title}")
            scrape_topic_to_text(driver, title, href)
            time.sleep(1)  # polite delay
    finally:
        driver.quit()

if __name__ == "__main__":
    scrape_all_latest(target_topics=200)


KeyboardInterrupt: 

In [11]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def init_driver(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(5)
    return driver

def scroll_and_collect_topics(driver, target_count=200, pause_time=2):
    """Scrolls the latest page to collect topic links."""
    driver.get("https://community.patient.info/c/heart-health-and-blood-vessels/18")
    topic_links = []

    while len(topic_links) < target_count:
        # Scroll to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)

        # Collect links
        topics = driver.find_elements(By.CSS_SELECTOR, "a.title.raw-topic-link")
        for topic in topics:
            href = topic.get_attribute("href")
            title = topic.text.strip()
            if href and (title, href) not in topic_links:
                topic_links.append((title, href))

    print(f"Collected {len(topic_links[:target_count])} topics")
    return topic_links[:target_count]

def scroll_to_load_posts(driver, target_count=9999, pause_time=1):
    """Scrolls inside a topic to load all posts."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    posts_loaded = len(driver.find_elements(By.CSS_SELECTOR, "div.topic-post"))

    while posts_loaded < target_count:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)
        posts_loaded = len(driver.find_elements(By.CSS_SELECTOR, "div.topic-post"))
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    print(f"Loaded {posts_loaded} posts in topic")

def scrape_topic_to_text(driver, title, topic_url, output_dir="output"):
    try:
        driver.get(topic_url)
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.cooked"))
        )
        scroll_to_load_posts(driver)
    except Exception as e:
        print(f"Failed to load topic {topic_url}: {e}")
        return

    # Get page title
    try:
        page_title = driver.find_element(By.CSS_SELECTOR, "h1.title").text.strip()
    except:
        page_title = title

    posts = []
    post_containers = driver.find_elements(By.CSS_SELECTOR, "div.topic-post")
    if not post_containers:
        post_containers = driver.find_elements(By.CSS_SELECTOR, "div.cooked")

    for container in post_containers:
        author = None
        timestamp = None
        content = None

        try:
            author = container.find_element(By.CSS_SELECTOR, ".creator a.username").text.strip()
        except:
            pass
        try:
            time_el = container.find_element(By.CSS_SELECTOR, "time")
            timestamp = time_el.get_attribute("datetime")
        except:
            pass
        try:
            content = container.find_element(By.CSS_SELECTOR, "div.cooked").text.strip()
        except:
            content = container.text.strip()

        posts.append({
            "author": author,
            "timestamp": timestamp,
            "content": content
        })

    # Save to .txt
    safe_title = "".join(c if c.isalnum() or c in " _-" else "_" for c in page_title)[:80]
    os.makedirs(output_dir, exist_ok=True)
    filepath = os.path.join(output_dir, f"{safe_title}.txt")

    with open(filepath, "w", encoding="utf-8") as f:
        f.write(f"Topic: {page_title}\nURL: {topic_url}\n\n")
        for idx, post in enumerate(posts, start=1):
            f.write(f"--- Post {idx} ---\n")
            f.write(f"Author: {post.get('author')}\n")
            f.write(f"Timestamp: {post.get('timestamp')}\n")
            f.write("Content:\n")
            f.write(post.get("content", "") + "\n\n")

    print(f"Wrote {len(posts)} posts to {filepath}")

def scrape_all_latest(target_topics=200, headless=False, start_from=None):
    driver = init_driver(headless=headless)
    try:
        topic_links = scroll_and_collect_topics(driver, target_count=target_topics)

        # If you know where to start, find its index
        if start_from:
            start_idx = 0
            for i, (title, _) in enumerate(topic_links):
                if start_from.lower() in title.lower():
                    start_idx = i
                    break
            topic_links = topic_links[start_idx:]
            print(f"🔁 Resuming from: {topic_links[0][0]}")
        else:
            print("🔁 Starting from the top.")

        for idx, (title, href) in enumerate(topic_links, start=1):
            # Skip if file already exists (useful for partial reruns)
            safe_title = "".join(c if c.isalnum() or c in " _-" else "_" for c in title)[:100]
            filepath = os.path.join("output", f"{safe_title}.txt")
            if os.path.exists(filepath):
                print(f"⏩ Skipping (already saved): {title}")
                continue

            print(f"\n📄 Scraping topic {idx}/{len(topic_links)}: {title}")
            scrape_topic_to_text(driver, title, href)
            time.sleep(1.5)
    finally:
        driver.quit()
        print("\n🎉 Done! Resumed scraping finished.")


if __name__ == "__main__":
    scrape_all_latest(
        target_topics=250,
        headless=False,
        start_from="My PVCs are causing crippling anxiety and panic attacks. Help!!!!"
    )



Collected 250 topics
🔁 Resuming from: My PVCs are causing crippling anxiety and panic attacks. Help!!!!

📄 Scraping topic 1/49: My PVCs are causing crippling anxiety and panic attacks. Help!!!!
Loaded 1 posts in topic
Wrote 1 posts to output\My PVCs are causing crippling anxiety and panic attacks_ Help____.txt

📄 Scraping topic 2/49: PVCs after every 2 beats, all day
Loaded 10 posts in topic
Wrote 10 posts to output\PVCs after every 2 beats_ all day.txt
⏩ Skipping (already saved): Pvcs

📄 Scraping topic 4/49: Heart
Loaded 2 posts in topic
Wrote 2 posts to output\Heart.txt

📄 Scraping topic 5/49: Arrhythmogenic cardiomyopathy
Loaded 1 posts in topic
Wrote 1 posts to output\Arrhythmogenic cardiomyopathy.txt

📄 Scraping topic 6/49: From my first SVT episode to my ablation
Loaded 3 posts in topic
Wrote 3 posts to output\From my first SVT episode to my ablation.txt

📄 Scraping topic 7/49: Is this normal after ablation
Loaded 1 posts in topic
Wrote 1 posts to output\Is this normal after abla